In [1]:
%pip install oracledb
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import oracledb
import pandas as pd

# Connect to the Oracle database
operational_connection = oracledb.connect(user="walmart", password="walmart", dsn="127.0.0.1:1521/freepdb1")
print("Connected to Oracle Database")
 
# Create a cursor to execute queries
operational_cursor = operational_connection.cursor()
def get_data_frame(cursor, query):
    # Execute the SQL query
    cursor.execute(query)
    # Fetch all rows from the query result
    data = cursor.fetchall()
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]
    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

Connected to Oracle Database


In [3]:
query='''SELECT c.CategoryID, p.ProductID, p.SupplierID,
 cu.CUSTOMERID, o.OrderID, od.ORDERDETAILID, o.TotalAmount,
  o.ORDERDATE, od.UNITPRICE, od.Quantity

FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
JOIN Suppliers s ON p.SupplierID = s.SupplierID
JOIN OrderDetails od ON p.ProductID = od.ProductID
JOIN Orders o ON od.OrderID = o.OrderID
JOIN Customers cu ON o.CustomerID = cu.CustomerID'''



In [5]:
fact_table = get_data_frame(operational_cursor, query)
fact_table.head()

,CATEGORYID,PRODUCTID,SUPPLIERID,CUSTOMERID,ORDERID,ORDERDETAILID,TOTALAMOUNT,ORDERDATE,UNITPRICE,QUANTITY
0,1,1,1,1,1,1,500.0,2025-01-01 10:00:00,50.0,3
1,2,2,2,2,2,2,700.0,2025-01-02 14:30:00,60.0,2
2,3,3,3,3,3,3,650.0,2025-01-03 09:00:00,70.0,1
3,4,4,4,4,4,4,800.0,2025-01-04 11:45:00,55.0,5
4,5,5,5,5,5,5,900.0,2025-01-05 16:15:00,60.0,4


In [6]:
# Connect to the Oracle database
star_connection = oracledb.connect(user="walmart_star_schema", password="walmart", dsn="127.0.0.1:1521/freepdb1")
star_cursor = star_connection.cursor()
print("Connected to Oracle Database")
def insert_data_frame(cursor, query, dataFrame):
    for i, row in dataFrame.iterrows():
        cursor.execute(query, tuple(row))
    cursor.execute('commit')

Connected to Oracle Database


In [7]:
# Load customer_dim data and put into a data frame. 
customer_dim_query = "SELECT  Row_ID, CustomerID FROM CUSTOMER_DIM"
customer_dim_df = get_data_frame(star_cursor, customer_dim_query)
customer_dim_df.head()

,ROW_ID,CUSTOMERID
0,1,86
1,2,87
2,3,88
3,4,89
4,5,90


In [16]:
merged_df = pd.merge(fact_table, customer_dim_df[['CUSTOMERID', 'ROW_ID']], 
                      left_on='CUSTOMERID', right_on='CUSTOMERID', how='inner')
merged_df.rename(columns={"ROW_ID": "Cus_id"}, inplace=True)
merged_df

,CATEGORYID,PRODUCTID,SUPPLIERID,CUSTOMERID,ORDERID,ORDERDETAILID,TOTALAMOUNT,ORDERDATE,UNITPRICE,QUANTITY,Cus_id
0,1,1,1,1,1,1,500.0,2025-01-01 10:00:00,50.0,3,16
1,1,1,1,1,1,1,500.0,2025-01-01 10:00:00,50.0,3,116
2,2,2,2,2,2,2,700.0,2025-01-02 14:30:00,60.0,2,17
3,2,2,2,2,2,2,700.0,2025-01-02 14:30:00,60.0,2,117
4,3,3,3,3,3,3,650.0,2025-01-03 09:00:00,70.0,1,18
...,...,...,...,...,...,...,...,...,...,...,...
155,4,26,6,78,78,78,700.0,2025-03-19 10:30:00,55.0,5,193
156,1,1,1,79,79,79,800.0,2025-03-20 11:45:00,60.0,3,94
157,1,1,1,79,79,79,800.0,2025-03-20 11:45:00,60.0,3,194
158,2,2,2,80,80,80,650.0,2025-03-21 09:15:00,65.0,2,95


In [21]:
# Load customer_dim data and put into a data frame. 
Category_dim_query = "SELECT  Row_ID, CategoryID FROM Category_DIM"
Category_dim_df = get_data_frame(star_cursor, Category_dim_query)
merged_df = pd.merge(merged_df, Category_dim_df[['CATEGORYID', 'ROW_ID']], 
                      left_on='CATEGORYID', right_on='CATEGORYID', how='inner')
merged_df.rename(columns={"ROW_ID": "Cat_id"}, inplace=True)
merged_df

,ROW_ID,CATEGORYID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [ ]:
# Load customer_dim data and put into a data frame. 
date_dim_query = "SELECT  Row_ID, TIME_STAMP FROM DATE_DIM"
date_dim_df = get_data_frame(star_cursor, date_dim_query)
merged_df = pd.merge(merged_df, date_dim_df[['TIME_STAMP', 'ROW_ID']], 
                      left_on='ORDERDATE', right_on='TIME_STAMP', how='inner')
merged_df.rename(columns={"ROW_ID": "Date_id"}, inplace=True)
merged_df

In [ ]:
# Load customer_dim data and put into a data frame. 
product_dim_query = "SELECT  Row_ID, PRODUCTID FROM PRODUCT_DIM"
Product_dim_df = get_data_frame(star_cursor, product_dim_query)
merged_df = pd.merge(merged_df, Product_dim_df[['PRODUCTID', 'ROW_ID']], 
                      left_on='PRODUCTID', right_on='PRODUCTID', how='inner')
merged_df.rename(columns={"ROW_ID": "Product_id"}, inplace=True)
merged_df

In [ ]:
# Load customer_dim data and put into a data frame. 
suppliers_dim_query = "SELECT  Row_ID, SUPPLIERID FROM SUPPLIERS_DIM"
suppliers_dim_df = get_data_frame(star_cursor, suppliers_dim_query)
merged_df = pd.merge(merged_df, suppliers_dim_df[['SUPPLIERID', 'ROW_ID']], 
                      left_on='SUPPLIERID', right_on='SUPPLIERID', how='inner')
merged_df.rename(columns={"ROW_ID": "supplier_id"}, inplace=True)
merged_df

In [47]:
final_df = merged_df[['Product_id','Date_id','Cus_id','Cat_id','supplier_id','ORDERID','QUANTITY','UNITPRICE','TOTALAMOUNT']]
final_df.head()

,Product_id,Date_id,Cus_id,Cat_id,supplier_id,ORDERID,QUANTITY,UNITPRICE,TOTALAMOUNT
0,1,1,16,1,1,1,3,50.0,500.0
1,1,1,116,1,1,1,3,50.0,500.0
2,2,2,17,2,2,2,2,60.0,700.0
3,2,2,117,2,2,2,2,60.0,700.0
4,3,3,18,3,3,3,1,70.0,650.0


In [49]:
final_df.rename(columns={'Product_id':'PRODUCTID',
                          'Date_id': 'DATEID',
                          'Cus_id': 'CUSTOMERID',
                          'Cat_id': 'CATEGORYID',
                          'supplier_id': 'SUPPLIERID'}, inplace=True)
final_df.head()

/var/folders/x5/l8f6fzld7cq3_hk2d62jqqkw0000gn/T/ipykernel_3268/3506157790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'Product_id':'PRODUCTID',


,PRODUCTID,DATEID,CUSTOMERID,CATEGORYID,SUPPLIERID,ORDERID,QUANTITY,UNITPRICE,TOTALAMOUNT
0,1,1,16,1,1,1,3,50.0,500.0
1,1,1,116,1,1,1,3,50.0,500.0
2,2,2,17,2,2,2,2,60.0,700.0
3,2,2,117,2,2,2,2,60.0,700.0
4,3,3,18,3,3,3,1,70.0,650.0


In [56]:
query = '''
    insert into TRANSACTION_FACT(PRODUCTID, DATEID, CUSTOMERID, CATEGORYID, SUPPLIERID,
       ORDERID, QUANTITY, UNITPRICE, TOTALAMOUNT) values(:1, :2, :3, :4, :5, :6, :7, :8, :9)
'''
insert_data_frame(star_cursor,query,final_df)